In [1]:
import os

from pyment.data import NiftiDataset, AsyncNiftiGenerator
from pyment.data.preprocessors import NiftiPreprocessor

ixi_folder = os.path.join(os.path.expanduser('~'), 'data', 'IXI', 'cropped')
dataset = NiftiDataset.from_folder(ixi_folder)
preprocessor = NiftiPreprocessor(sigma=255.)
generator = AsyncNiftiGenerator(
    dataset=dataset,
    preprocessor=preprocessor,
    batch_size=4,
    threads=8
)

2022-03-22 11:07:00,805 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI589-Guys-1080-T1: Missing labels
2022-03-22 11:07:00,805 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI345-IOP-0928-T1: Missing labels
2022-03-22 11:07:00,806 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI228-Guys-0822-T1: Missing labels
2022-03-22 11:07:00,806 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI341-Guys-0906-T1: Missing labels
2022-03-22 11:07:00,806 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI347-IOP-0927-T1: Missing labels
2022-03-22 11:07:00,807 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI337-IOP-0929-T1: Missing labels
2022-03-22 11:07:00,807 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI457-Guys-0984-T1: Missing labels
2022-03-22 11:07:00,808 - WARNING - pyment.data.datasets.nifti_dataset: Skipping IXI088-Guys-0758-T1: Missing labels
2022-03-22 11:07:00,808 - WARNING - pyment.data.datasets.nifti_data

In [2]:


from pyment.models import RegressionSFCN

model = RegressionSFCN(weights='brain-age')

predictions = model.predict(generator)

2022-03-22 11:07:01.695008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 11:07:01.702393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 11:07:01.702723: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 11:07:01.703551: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

(None, 1)
3


  0%|                                                                                                                                                                                                                                                                                               | 0/135 [00:00<?, ?it/s]2022-03-22 11:07:02.879489: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-22 11:07:03.379123: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100
2022-03-22 11:07:03.596768: W tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
100%|████████████████████████████████████████████████████████████

In [11]:
import numpy as np


ages = dataset['age']
predictions = predictions.squeeze()
predictions = predictions[np.where(~np.isnan(ages))]
ages = ages[np.where(~np.isnan(ages))]
delta = predictions - ages
print(f'Brain age delta: {round(np.mean(np.abs(delta)), 2)}')

Brain age delta: 4.0


In [14]:
from explainability import LRP


lrp = LRP(model, layer=29, idx=0, alpha=2, beta=1)

NotImplementedError: LRP is not implemented for layer Add

In [12]:
for i in range(len(model.layers)):
    print(f'{i}: {model.layers[i]}')

0: <keras.engine.input_layer.InputLayer object at 0x7fd403f68ed0>
1: <keras.layers.core.Reshape object at 0x7fd40269aad0>
2: <keras.layers.convolutional.Conv3D object at 0x7fd40269abd0>
3: <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fd401e22a50>
4: <keras.layers.core.Activation object at 0x7fd401e07c10>
5: <keras.layers.pooling.MaxPooling3D object at 0x7fd401d7f250>
6: <keras.layers.convolutional.Conv3D object at 0x7fd401d5d290>
7: <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fd401d5dd10>
8: <keras.layers.core.Activation object at 0x7fd401d694d0>
9: <keras.layers.pooling.MaxPooling3D object at 0x7fd401d6f390>
10: <keras.layers.convolutional.Conv3D object at 0x7fd401d74a10>
11: <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fd401d6fc90>
12: <keras.layers.core.Activation object at 0x7fd401d01090>
13: <keras.layers.pooling.MaxPooling3D object at 0x7fd401d09650>
14: <keras.layers.convolu